Nhóm 18
- Đỗ Tấn Lực - 23520903
- Lê Quang Long - -23520878

#### Import các thư viện cần thiết cho xử lý dữ liệu, xây dựng mô hình, và Spark

- pandas, numpy: Xử lý dữ liệu.
- torch, torch.nn, torch.optim: Xây dựng và huấn luyện mô hình deep learning.
- sklearn: Tiền xử lý, chia tập, đánh giá mô hình.
- re: Xử lý chuỗi, regex.
- kagglehub: Tải dataset từ Kaggle.
- pyspark: Xử lý dữ liệu lớn với Spark, định nghĩa UDF.
- transformers: Sử dụng mô hình BERT (PhoBERT).

In [ ]:
import pandas as pd # Thư viện xử lý dữ liệu dạng bảng
import numpy as np # Thư viện xử lý mảng và tính toán số học
import torch # Thư viện PyTorch để xây dựng và huấn luyện mô hình deep learning
import torch.nn as nn # Module chứa các lớp mạng nơ-ron
import torch.optim as optim # Module chứa các thuật toán tối ưu (Adam, SGD,...)
from torch.utils.data import Dataset, DataLoader # Các lớp để xử lý dữ liệu và tạo DataLoader
from sklearn.model_selection import train_test_split # Hàm chia dữ liệu thành tập train/test/val
from sklearn.preprocessing import LabelEncoder # Công cụ để mã hóa nhãn
from sklearn.metrics import accuracy_score # Hàm tính độ chính xác
from collections import Counter # Công cụ đếm tần suất
import re # Thư viện xử lý biểu thức chính quy (regex)
import kagglehub # Thư viện tải dataset từ Kaggle
from pyspark.sql import SparkSession # Lớp tạo SparkSession để làm việc với Spark SQL
from pyspark.sql.types import DoubleType, ArrayType, StringType, IntegerType # Các kiểu dữ liệu của Spark SQL
from pyspark.sql.functions import pandas_udf, PandasUDFType # Hàm định nghĩa UDF cho Spark
from pyspark.ml.functions import predict_batch_udf # Hàm dự đoán theo batch với Spark UDF
from pyspark.ml.evaluation import MulticlassClassificationEvaluator # Công cụ đánh giá mô hình phân loại đa lớp trong Spark ML
from transformers import AutoTokenizer, AutoModel # Các lớp để sử dụng mô hình từ thư viện transformers (ví dụ: BERT)

#### Tải dataset ABSA tiếng Việt từ Kaggle

- Sử dụng kagglehub để tải về bộ dữ liệu.
- Đọc các file train, test, val bằng pandas.

In [ ]:
path = kagglehub.dataset_download("cthng123/absa-vietnamese") # Tải dataset từ Kaggle Hub và lấy đường dẫn

train = pd.read_csv(f"{path}/train_data.csv") # Đọc file train_data.csv vào DataFrame pandas
test = pd.read_csv(f"{path}/test_data.csv") # Đọc file test_data.csv vào DataFrame pandas
val = pd.read_csv(f"{path}/val_data.csv") # Đọc file val_data.csv vào DataFrame pandas

### Tiền xử lý dữ liệu văn bản

Ở bước này, chúng ta sẽ làm sạch dữ liệu review bằng cách loại bỏ emoji, ký tự đặc biệt, chuyển về chữ thường và tách từ. Đây là bước quan trọng giúp chuẩn hóa dữ liệu đầu vào cho mô hình học máy.

#### 1. Tạo regex để loại bỏ emoji và ký tự đặc biệt

Đoạn regex dưới đây giúp nhận diện và loại bỏ các emoji, biểu tượng, ký tự đặc biệt thường xuất hiện trong văn bản mạng xã hội, đánh giá sản phẩm, v.v.

**Tạo regex để loại bỏ emoji và ký tự đặc biệt:**

In [ ]:
# Định nghĩa biểu thức chính quy (regex) để tìm và loại bỏ các ký tự emoji và biểu tượng đặc biệt.
# Biểu thức này bao gồm nhiều khoảng mã Unicode đại diện cho các loại emoji khác nhau.
emoji = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u'\U00010000-\U0010ffff'
    u"\u200d"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\u3030"
    u"\ufe0f"
"]+", flags=re.UNICODE)

## - Đoạn regex này giúp nhận diện và loại bỏ các emoji, biểu tượng, ký tự đặc biệt thường xuất hiện trong văn bản mạng xã hội, đánh giá sản phẩm, v.v.

#### 2. Định nghĩa hàm tiền xử lý văn bản

Hàm `preprocess` sẽ:
- Loại bỏ emoji bằng regex ở trên.
- Loại bỏ ký tự đặc biệt, chỉ giữ lại chữ cái, số và khoảng trắng.
- Chuyển toàn bộ về chữ thường.
- Chuẩn hóa khoảng trắng.
- Tách từ thành list (tokens).

In [ ]:
def preprocess(text):
  if not isinstance(text, str):
    return text
  text = re.sub(emoji, '', text) # Loại bỏ emoji từ văn bản
  text = re.sub(r"[^\w\s]", "", text)  # Loại bỏ ký tự đặc biệt, chỉ giữ lại chữ và số
  text = text.lower() # Chuyển toàn bộ văn bản về chữ thường
  text = re.sub(r"\s+", " ", text).strip() # Chuẩn hóa khoảng trắng (thay thế nhiều khoảng trắng bằng một khoảng trắng duy nhất và loại bỏ khoảng trắng ở đầu/cuối)
  return text.split() # Tách văn bản thành danh sách các từ (tokens)

#### 3. Áp dụng hàm tiền xử lý lên dữ liệu

Áp dụng hàm `preprocess` cho từng review trong tập train, test, val để thu được danh sách token.

In [ ]:
# Áp dụng hàm preprocess lên cột 'Review' của từng tập dữ liệu để tạo cột 'tokens' chứa danh sách các từ đã tiền xử lý
train['tokens'] = train['Review'].apply(preprocess)
test['tokens'] = test['Review'].apply(preprocess)
val['tokens'] = val['Review'].apply(preprocess)

# Gom tất cả token từ tập train lại để xây dựng từ điển
all_tokens = [token for tokens in train['tokens'] for token in tokens]
all_tokens # Hiển thị danh sách tất cả các token

['giày',
 'đẹp',
 'đi',
 'êm',
 'lắm',
 'mình',
 'săn',
 'sale',
 'với',
 'giá',
 'khá',
 'rẻ',
 'chất',
 'lượng',
 'ok',
 'shipper',
 'thân',
 'thiện',
 'sẽ',
 'tiếp',
 'tục',
 'ủng',
 'hộ',
 'shop',
 'hình',
 'ảnh',
 'và',
 'video',
 'chỉ',
 'mang',
 'tính',
 'chất',
 'minh',
 'họa',
 'thôi',
 'về',
 'sản',
 'phẩm',
 'tính',
 'thang',
 'điểm',
 'thì',
 '9',
 '10',
 'ạ',
 'khá',
 'tốt',
 'hợp',
 'với',
 'giá',
 'tiền',
 'shipper',
 'thì',
 'ân',
 'cần',
 'thân',
 'thiện',
 'giao',
 'hàng',
 'khá',
 'nhanh',
 'ạ',
 'mình',
 'đặt',
 'size',
 '39',
 'nhưng',
 'chật',
 'k',
 'đeo',
 'nổi',
 'còn',
 'giầy',
 'thì',
 'đẹp',
 'giao',
 'hàng',
 'nhanh',
 'nên',
 'mua',
 'nha',
 'mọi',
 'người',
 'đẹp',
 'xuất',
 'sắc',
 'lun',
 'ạ',
 'giày',
 'đẹp',
 'giống',
 'hình',
 'mọi',
 'người',
 'nên',
 'mua',
 'tăng',
 '1',
 'size',
 'để',
 'khỏi',
 'phải',
 'mang',
 'chật',
 'chân',
 'đáng',
 'mua',
 'mọi',
 'người',
 'nên',
 'mua',
 'giao',
 'hàng',
 'nhanh',
 'đặt',
 'đế',
 'đen',
 'giao',
 'đế',


#### 4. Xây dựng từ điển (vocab) và ánh xạ từ sang chỉ số

- Đếm tần suất xuất hiện của từng token.
- Chỉ giữ lại các token xuất hiện nhiều hơn 1 lần, thêm token đặc biệt `<pad>`, `<unk>`.
- Tạo ánh xạ từ sang chỉ số (word_to_idx).

In [ ]:
# Đếm tần suất xuất hiện của từng token trong danh sách all_tokens
vocab_counts = Counter(all_tokens)
# Xây dựng từ điển (vocab): bắt đầu với các token đặc biệt '<pad>' (cho padding) và '<unk>' (cho các từ không xác định),
# sau đó thêm các từ xuất hiện với tần suất lớn hơn 1
vocab = ['<pad>', '<unk>'] + [word for word, fre in vocab_counts.most_common() if fre > 1]
# Tạo ánh xạ từ từ (word) sang chỉ số (idx) trong từ điển
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
word_to_idx # Hiển thị từ điển ánh xạ

Counter({'giày': 4699,
         'đẹp': 4045,
         'đi': 1838,
         'êm': 693,
         'lắm': 1532,
         'mình': 1294,
         'săn': 98,
         'sale': 106,
         'với': 1315,
         'giá': 1958,
         'khá': 682,
         'rẻ': 783,
         'chất': 1687,
         'lượng': 941,
         'ok': 1012,
         'shipper': 190,
         'thân': 225,
         'thiện': 196,
         'sẽ': 863,
         'tiếp': 290,
         'tục': 79,
         'ủng': 775,
         'hộ': 827,
         'shop': 2364,
         'hình': 980,
         'ảnh': 829,
         'và': 965,
         'video': 165,
         'chỉ': 608,
         'mang': 1278,
         'tính': 587,
         'minh': 165,
         'họa': 62,
         'thôi': 202,
         'về': 397,
         'sản': 921,
         'phẩm': 915,
         'thang': 9,
         'điểm': 136,
         'thì': 1221,
         '9': 11,
         '10': 121,
         'ạ': 978,
         'tốt': 642,
         'hợp': 623,
         'tiền': 845,
         'ân':

#### 5. Chuyển tokens thành chỉ số và padding

- Chuyển mỗi danh sách token thành danh sách chỉ số (index).
- Padding hoặc cắt bớt về độ dài cố định (MAX_LEN).

In [ ]:
# Hàm chuyển đổi danh sách các token thành danh sách các chỉ số tương ứng trong từ điển.
# Nếu một token không có trong từ điển, nó sẽ được gán chỉ số của '<unk>' (là 1).
def tokens_to_ids(tokens):
    return [word_to_idx.get(token, 1) for token in tokens]  # 1 is <unk>

# Áp dụng hàm tokens_to_ids cho cột 'tokens' của từng tập dữ liệu để tạo cột 'input_ids'
train['input_ids'] = train['tokens'].apply(tokens_to_ids)
val['input_ids'] = val['tokens'].apply(tokens_to_ids)
test['input_ids'] = test['tokens'].apply(tokens_to_ids)

MAX_LEN = 128 # Định nghĩa độ dài cố định cho chuỗi đầu vào
# Hàm padding hoặc cắt bớt chuỗi chỉ số để đảm bảo tất cả có cùng độ dài MAX_LEN.
# Nếu chuỗi ngắn hơn MAX_LEN, thêm các chỉ số 0 (tương ứng với '<pad>') vào cuối.
# Nếu chuỗi dài hơn MAX_LEN, cắt bớt phần cuối.
def pad_sequence(seq):
    return seq[:MAX_LEN] + [0] * max(0, MAX_LEN - len(seq))

# Áp dụng hàm pad_sequence cho cột 'input_ids' của từng tập dữ liệu
train['input_ids'] = train['input_ids'].apply(pad_sequence)
val['input_ids'] = val['input_ids'].apply(pad_sequence)
test['input_ids'] = test['input_ids'].apply(pad_sequence)

#### 6. Chuyển đổi nhãn (label) về dạng số

- Các nhãn gốc của từng khía cạnh (aspect) là: -1, 0, 1, 2.
- Ta ánh xạ các giá trị này về các số 0, 1, 2, 3 để phù hợp với CrossEntropyLoss của PyTorch.
- Thực hiện chuyển đổi cho cả train, val, test.

In [ ]:
ASPECTS = ['Price', 'Shipping', 'Outlook', 'Quality', 'Size', 'Shop_Service', 'General', 'Others'] # Danh sách các khía cạnh (aspects)
NUM_CLASSES = 4 # Số lượng lớp (sentiment: -1, 0, 1, 2)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Chọn thiết bị (GPU hoặc CPU) để chạy mô hình

label_map = {-1: 0, 0: 1, 1: 2, 2: 3} # Ánh xạ các nhãn gốc (-1, 0, 1, 2) sang chỉ số (0, 1, 2, 3)

# Lấy các cột nhãn từ DataFrame train và chuyển đổi sang mảng numpy
train_labels_raw = np.array(train[ASPECTS])
# Áp dụng ánh xạ label_map cho từng giá trị trong mảng nhãn, gán giá trị mặc định là 0 nếu không tìm thấy trong map
train_labels = np.vectorize(lambda x: label_map.get(x, 0))(train_labels_raw)  # Default to 0 if unexpected

# Tương tự cho tập test
test_labels_raw = np.array(test[ASPECTS])
test_labels = np.vectorize(lambda x: label_map.get(x, 0))(test_labels_raw)

# Tương tự cho tập validation
val_labels_raw = np.array(val[ASPECTS])
val_labels = np.vectorize(lambda x: label_map.get(x, 0))(val_labels_raw)

#### 7. Định nghĩa Dataset cho PyTorch

- Tạo class kế thừa `torch.utils.data.Dataset` để quản lý dữ liệu đầu vào và nhãn.
- Chuẩn bị DataLoader cho train, val, test.

In [ ]:
# Định nghĩa lớp Dataset tùy chỉnh cho PyTorch, kế thừa từ torch.utils.data.Dataset
class ABSADataset(Dataset):
    # Hàm khởi tạo: nhận vào dữ liệu đầu vào (input_ids) và nhãn
    def __init__(self, inputs, labels):
        # Chuyển dữ liệu đầu vào sang tensor PyTorch với dtype long
        self.inputs = torch.tensor(inputs.values.tolist(), dtype=torch.long)
        # Chuyển nhãn sang tensor PyTorch với dtype long
        self.labels = torch.tensor(labels, dtype=torch.long)

    # Hàm trả về số lượng mẫu trong dataset
    def __len__(self):
        return len(self.inputs)

    # Hàm trả về một cặp (dữ liệu đầu vào, nhãn) tại chỉ số idx
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

# Tạo các instance của ABSADataset cho tập train, test, và validation
train_data = ABSADataset(train['input_ids'], train_labels)
test_data = ABSADataset(test['input_ids'], test_labels)
val_data = ABSADataset(val['input_ids'], val_labels)

# Tạo DataLoader cho từng tập dữ liệu để load dữ liệu theo batch trong quá trình huấn luyện và đánh giá
train_load = DataLoader(train_data, batch_size=32, shuffle=True) # Shuffle=True cho tập train để xáo trộn dữ liệu
test_load = DataLoader(test_data, batch_size=32)
val_load = DataLoader(val_data, batch_size=32)

#### 8. Định nghĩa mô hình LSTM cho ABSA

- Mô hình gồm các lớp: Embedding, LSTM 2 chiều, Dropout, Fully Connected.
- Đầu ra được reshape về (batch_size, num_aspects, num_classes).

In [ ]:
# Định nghĩa mô hình LSTM cho ABSA, kế thừa từ torch.nn.Module
class LSTM_ABSA(nn.Module):
    # Hàm khởi tạo với các tham số: kích thước từ điển, kích thước embedding, kích thước lớp ẩn LSTM, số lượng khía cạnh, số lượng lớp sentiment, tỷ lệ dropout
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256, num_aspects=len(ASPECTS), num_classes=NUM_CLASSES, dropout_prob=0.5):
        super().__init__() # Gọi hàm khởi tạo của lớp cha
        # Lớp Embedding: chuyển chỉ số từ thành vector mật độ cao
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) # padding_idx=0 để bỏ qua padding
        # Lớp LSTM hai chiều: xử lý chuỗi embedding
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True) # bidirectional=True sử dụng LSTM hai chiều
        # Lớp Dropout: áp dụng dropout để chống overfitting
        self.dropout = nn.Dropout(dropout_prob)
        # Lớp Fully Connected: ánh xạ đầu ra của LSTM đến không gian đầu ra (số khía cạnh * số lớp)
        self.fc = nn.Linear(hidden_dim * 2, num_aspects * num_classes) # hidden_dim * 2 vì là LSTM hai chiều
        self.num_classes = num_classes # Lưu lại số lượng lớp
        self.num_aspects = num_aspects # Lưu lại số lượng khía cạnh

    # Hàm forward: định nghĩa luồng dữ liệu đi qua mô hình
    def forward(self, x):
        embedding = self.embedding(x) # Áp dụng lớp Embedding
        _, (hn, _) = self.lstm(embedding) # Áp dụng lớp LSTM, chỉ lấy hidden state cuối cùng (hn)
        hn = torch.cat((hn[0], hn[1]), dim=1) # Nối hidden state của hai chiều
        hn = self.dropout(hn) # Áp dụng Dropout
        out = self.fc(hn) # Áp dụng lớp Fully Connected
        # Reshape đầu ra về kích thước (batch_size, num_aspects, num_classes)
        return out.view(-1, self.num_aspects, self.num_classes)

# Khởi tạo mô hình LSTM, chuyển nó đến thiết bị đã chọn (GPU/CPU)
model_lstm = LSTM_ABSA(len(vocab)).to(DEVICE)
# Định nghĩa hàm mất mát (Loss function) là CrossEntropyLoss, phù hợp cho bài toán phân loại đa lớp
criterion = nn.CrossEntropyLoss()
# Định nghĩa thuật toán tối ưu (Optimizer) là Adam
optimizer = optim.Adam(model_lstm.parameters(), lr=0.001) # Tham số model_lstm.parameters() để tối ưu hóa các trọng số của mô hình

#### 9. Huấn luyện và đánh giá mô hình LSTM

- Huấn luyện mô hình trên tập train, đánh giá trên tập validation.
- Lưu lại trọng số mô hình sau khi huấn luyện.

In [ ]:
epoch = 1 # Số lượng epoch để huấn luyện
for e in range(epoch):
    model_lstm.train() # Chuyển mô hình sang chế độ huấn luyện (bật dropout, batch norm)
    total_loss = 0 # Khởi tạo tổng mất mát cho epoch
    for input, label in train_load: # Lặp qua từng batch dữ liệu trong DataLoader train
        input, label = input.to(DEVICE), label.to(DEVICE) # Chuyển dữ liệu đầu vào và nhãn sang thiết bị đã chọn
        optimizer.zero_grad() # Đặt lại gradient về 0 trước khi tính toán backpropagation
        output = model_lstm(input) # Thực hiện forward pass để lấy đầu ra dự đoán
        # Tính mất mát cho từng khía cạnh và cộng lại
        loss = sum(criterion(output[:, i, :], label[:, i]) for i in range(len(ASPECTS)))
        loss.backward() # Thực hiện backpropagation để tính gradient
        optimizer.step() # Cập nhật trọng số của mô hình dựa trên gradient
        total_loss += loss.item() # Cộng mất mát của batch vào tổng mất mát

    # In mất mát trung bình của epoch
    print(f"Epoch {e+1}/{epoch}, Loss: {total_loss/len(train_load)}")

# Đánh giá mô hình trên tập validation
model_lstm.eval() # Chuyển mô hình sang chế độ đánh giá (tắt dropout, batch norm)
val_pred = [] # Danh sách lưu trữ các dự đoán trên tập validation
val_true = [] # Danh sách lưu trữ các nhãn thực tế trên tập validation
val_loss = 0 # Khởi tạo tổng mất mát cho tập validation
with torch.no_grad(): # Không tính toán gradient trong quá trình đánh giá để tiết kiệm bộ nhớ và tăng tốc độ
    for inputs, labels in val_load: # Lặp qua từng batch dữ liệu trong DataLoader validation
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE) # Chuyển dữ liệu đầu vào và nhãn sang thiết bị đã chọn
        outputs = model_lstm(inputs) # Thực hiện forward pass
        # Tính mất mát cho từng khía cạnh và cộng lại
        val_loss += sum(criterion(outputs[:, i, :], labels[:, i]).item() for i in range(len(ASPECTS)))
        # Lấy chỉ số của lớp có xác suất cao nhất làm dự đoán và chuyển về numpy array
        val_pred.extend(torch.argmax(outputs, dim=2).cpu().numpy())
        # Chuyển nhãn thực tế về numpy array
        val_true.extend(labels.cpu().numpy())

val_loss /= len(val_load) # Tính mất mát trung bình trên tập validation
# Tính độ chính xác trên tập validation bằng cách làm phẳng mảng dự đoán và nhãn thực tế
val_acc = accuracy_score(np.array(val_true).flatten(), np.array(val_pred).flatten())
print(f'Epoch {epoch+1}, Val Loss: {val_loss}, Val Accuracy: {val_acc}') # In kết quả đánh giá

# Lưu trọng số của mô hình sau khi huấn luyện
torch.save(model_lstm.state_dict(), 'lstm_absa_model.pth')

#### 10. Đánh giá mô hình trên tập test

- Dự đoán và tính độ chính xác trên tập test.

In [ ]:
model_lstm.eval() # Chuyển mô hình sang chế độ đánh giá
test_preds = [] # Danh sách lưu trữ các dự đoán trên tập test
test_true = [] # Danh sách lưu trữ các nhãn thực tế trên tập test
with torch.no_grad(): # Không tính toán gradient
    for inputs, labels in test_load: # Lặp qua từng batch dữ liệu trong DataLoader test
        inputs = inputs.to(DEVICE) # Chuyển dữ liệu đầu vào sang thiết bị đã chọn
        outputs = model_lstm(inputs) # Thực hiện forward pass
        # Lấy chỉ số của lớp có xác suất cao nhất làm dự đoán và chuyển về numpy array
        test_preds.extend(torch.argmax(outputs, dim=2).cpu().numpy())
        # Chuyển nhãn thực tế về numpy array
        test_true.extend(labels.cpu().numpy())

#### 11. Dự đoán song song với Spark predict_batch_udf (LSTM)

- Sử dụng Spark để dự đoán song song theo batch trên tập test với mô hình đã huấn luyện.
- Trả về dự đoán cho từng review dưới dạng mảng số.
- Đánh giá lại độ chính xác trên Spark DataFrame.

In [ ]:
# Khởi tạo SparkSession với tên ứng dụng 'ABSA_PredictBatchUDF'
spark = SparkSession.builder.appName('ABSA_PredictBatchUDF').getOrCreate()
# Tạo Spark DataFrame từ cột 'Review' của DataFrame pandas test
spark_test_df = spark.createDataFrame(test[['Review']])

# Định nghĩa hàm UDF để dự đoán theo batch
def predict_udf():
    # Hàm dự đoán chính, nhận vào một mảng numpy chứa các review
    def predict_absa_udf(Reviews: np.ndarray) -> np.ndarray:
        # Tải mô hình đã huấn luyện
        loaded_model = LSTM_ABSA(len(vocab)).to('cpu') # Khởi tạo mô hình mới trên CPU
        loaded_model.load_state_dict(torch.load('lstm_absa_model.pth')) # Tải trọng số đã lưu
        loaded_model.eval() # Chuyển mô hình sang chế độ đánh giá

        # Tiền xử lý dữ liệu đầu vào tương tự như khi huấn luyện
        tokens = [preprocess(r) for r in Reviews] # Áp dụng hàm preprocess cho từng review
        input_ids = [pad_sequence(tokens_to_ids(t)) for t in tokens] # Chuyển token thành chỉ số và padding
        inputs = torch.tensor(input_ids, dtype=torch.long) # Chuyển sang tensor PyTorch

        with torch.no_grad(): # Không tính toán gradient
            outputs = loaded_model(inputs) # Thực hiện forward pass để lấy đầu ra dự đoán
            preds = torch.argmax(outputs, dim=2).numpy() # Lấy chỉ số lớp có xác suất cao nhất và chuyển về numpy array

        return preds # Trả về mảng numpy chứa các dự đoán

    return predict_absa_udf # Trả về hàm UDF

# Định nghĩa Spark UDF sử dụng predict_batch_udf
# predict_udf(): hàm Python chứa logic dự đoán
# return_type: kiểu dữ liệu trả về của UDF (mảng số nguyên)
# batch_size: kích thước batch để xử lý song song
# input_tensor_shapes: định nghĩa hình dạng của tensor đầu vào (ở đây là chuỗi, nên [1] cho mỗi phần tử)
absa_batch_udf = predict_batch_udf(
    predict_udf,
    return_type=ArrayType(IntegerType()),
    batch_size=32,
    input_tensor_shapes=[[1]]
)

# Áp dụng UDF vào Spark DataFrame để tạo cột 'predictions'
result_df_udf = spark_test_df.withColumn('predictions', absa_batch_udf(spark_test_df['Review']))
# Hiển thị kết quả dự đoán
result_df_udf.show(truncate=False)

# Tạo Spark DataFrame từ kết quả dự đoán và nhãn thực tế trên tập test (từ bước 10)
preds_df = spark.createDataFrame(pd.DataFrame({'prediction': np.array(test_preds).flatten(), 'label': np.array(test_true).flatten()}))
# Chuyển kiểu dữ liệu của cột 'prediction' và 'label' sang DoubleType để sử dụng MulticlassClassificationEvaluator
preds_df = preds_df.withColumn('prediction', preds_df['prediction'].cast(DoubleType()))
preds_df = preds_df.withColumn('label', preds_df['label'].cast(DoubleType()))

# Khởi tạo công cụ đánh giá MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
# Tính độ chính xác trên Spark DataFrame
accuracy = evaluator.evaluate(preds_df)
print('Spark Test Accuracy:', accuracy) # In độ chính xác

25/10/07 07:32:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+
|Review                                                                                                                                                                                                                                               |predictions             |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+
|Giày hơi có mùi nồng, lưu ý đôi LA không phải đế xám nên mng cân nhắc kĩ nhé ạ nhma với giá tiền này thì là oke                                                                     